In [ ]:
# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
import torch

TORCH, CUDA = torch.__version__.split('+')

!pip install torch-scatter -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric gdown sklearn matplotlib networkx rich

In [ ]:
import numpy as np
import os
import gdown
from torch_geometric.data import InMemoryDataset, Data, Batch

class CosmicRayDS(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ["cr_sphere.npz"]

    @property
    def processed_file_names(self):
        return ["data.pt"]

    def download(self):
        url = "https://drive.google.com/u/0/uc?export=download&confirm=HgGH&id=1XKN-Ik7BDyMWdQ230zWS2bNxXL3_9jZq"
        if os.path.exists(self.raw_file_names[0]) == False:
            gdown.download(url, self.raw_file_names[0], quiet=True)

    def process(self):
        f = np.load(self.raw_file_names[0])
        x = torch.tensor(f["data"]).float()
        y = torch.tensor(f["label"]).float()
        data_list = []
        for idx in range(len(x)):
            data_list.append(
                Data(x=x[idx, :, 3].reshape(-1, 1), pos=x[idx, :, :3], y=y[idx])
            )
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

ds = CosmicRayDS(".")

 # Graph Handling
 For this exercise we will learn to use `pytorch_geometric` (PyG) to run GNNs.
 The library comes with a comprehensive [documentation](https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html) and not only provides tools to handle graphs but also provides a large set of GNN specific layers and dataset.

 ## Task 1.1
 [Data Handling of Graphs](https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html#data-handling-of-graphs) offers a nice introduction into handling graphs.
 Provide an adjacency matric for cyclic graph (each nodes connects to the next) with 5 nodes. Convert the adjacency matric to an edge_index using `dense_to_sparse`. With this edge_index implement a graph with features [0,1,...,n-1]:

In [ ]:
import torch
from torch_geometric.data import Data, Batch
from torch_geometric.utils import dense_to_sparse

adj = torch.tensor(
    [
        [0.0, 1.0, ?, ?, ?],
        ...
    ]
)

cycle = Data(
    x=...
    edge_index=...dense_to_sparse(...)[0],
)


 ## Task 1.2
 Implement binary tree with 7 nodes over three levels directly constructing the edge_index.:
    0
  1   2
 3 4 5 6

In [ ]:
tree = Data(
    x=...,
    edge_index=torch.tensor([[0, 1], ...]).T,
)

 ## Task 1.3
 Convert the implemented PyG graphs to `networkx` graphs and plot them.

In [ ]:
from torch_geometric.utils import to_networkx
import networkx as nx

nx.draw_kamada_kawai(...)

In [ ]:
nx.draw_kamada_kawai(...)

 ## Task 1.4
 Have a look at the documentation on [Mini-batches](https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html#mini-batches).

 Batch the two graphs together and plot the batch with networkx. What do you see ?

In [ ]:
batch = Batch.from_data_list(...)
...

 ## Task 1.5
 Inspect the properties of the batch.
 You can use the `inspect` method of the rich library or a simple `print`.
 What do the `batch` and `ptr` attributes do?

In [ ]:
import rich
rich.inspect(batch)

 Adapted from https://pytorch-geometric.readthedocs.io/en/latest/notes/colabs.html

 # 2 Introduction: Hands-on Graph Neural Networks

 Recently, deep learning on graphs has emerged to one of the hottest research fields in the deep learning community.
 Here, **Graph Neural Networks (GNNs)** aim to generalize classical deep learning concepts to irregular structured data (in contrast to images or texts) and to enable neural networks to reason about objects and their relations.

 This is done by following a simple **neural message passing scheme**, where node features $\mathbf{x}_v^{(\ell)}$ of all nodes $v \in \mathcal{V}$ in a graph $\mathcal{G} = (\mathcal{V}, \mathcal{E})$ are iteratively updated by aggregating localized information from their neighbors $\mathcal{N}(v)$:

 $$
 \mathbf{x}_v^{(\ell + 1)} = f^{(\ell + 1)}_{\theta} \left( \mathbf{x}_v^{(\ell)}, \left\{ \mathbf{x}_w^{(\ell)} : w \in \mathcal{N}(v) \right\} \right)
 $$

 This tutorial will introduce you to some fundamental concepts regarding deep learning on graphs via Graph Neural Networks based on the **[PyTorch Geometric (PyG) library](https://github.com/rusty1s/pytorch_geometric)**.
 PyTorch Geometric is an extension library to the popular deep learning framework [PyTorch](https://pytorch.org/), and consists of various methods and utilities to ease the implementation of Graph Neural Networks.

 Following [Kipf et al. (2017)](https://arxiv.org/abs/1609.02907), let's dive into the world of GNNs by looking at a simple graph-structured example, the well-known [**Zachary's karate club network**](https://en.wikipedia.org/wiki/Zachary%27s_karate_club). This graph describes a social network of 34 members of a karate club and documents links between members who interacted outside the club. Here, we are interested in detecting communities that arise from the member's interaction.

 PyTorch Geometric provides an easy access to this dataset via the [`torch_geometric.datasets`](https://pytorch-geometric.readthedocs.io/en/latest/modules/datasets.html#torch_geometric.datasets) subpackage:

In [ ]:
from torch_geometric.datasets import KarateClub

dataset = KarateClub()
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

 After initializing the [`KarateClub`](https://pytorch-geometric.readthedocs.io/en/latest/modules/datasets.html#torch_geometric.datasets.KarateClub) dataset, we first can inspect some of its properties.
 For example, we can see that this dataset holds exactly **one graph**, and that each node in this dataset is assigned a **34-dimensional feature vector** (which uniquely describes the members of the karate club).
 Furthermore, the graph holds exactly **4 classes**, which represent the community each node belongs to.

 Let's now look at the underlying graph in more detail:

In [ ]:
data = dataset[0]  # Get the first graph object.

print(data)
print('==============================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

 Each graph in PyTorch Geometric is represented by a single [`Data`](https://pytorch-geometric.readthedocs.io/en/latest/modules/data.html#torch_geometric.data.Data) object, which holds all the information to describe its graph representation.
 We can print the data object anytime via `print(data)` to receive a short summary about its attributes and their shapes:
 ```
 Data(edge_index=[2, 156], x=[34, 34], y=[34], train_mask=[34])
 ```
 We can see that this `data` object holds 4 attributes:
 1. The `edge_index` property holds the information about the **graph connectivity**, *i.e.*, a tuple of source and destination node indices for each edge.
 2. The **node features** `x` (each of the 34 nodes is assigned a 34-dim feature vector)
 3. The **node labels**  `y` (each node is assigned to exactly one class).
 4. There also exists an additional attribute called `train_mask`, which describes for which nodes we already know their community assigments.
 In total, we are only aware of the ground-truth labels of 4 nodes (one for each community), and the task is to infer the community assignment for the remaining nodes.

 ## Task 2.1
 Plot the network graph.
 Color the nodes according to their class.

In [ ]:
from torch_geometric.utils import to_networkx
import networkx as nx

nx.draw_networkx( ..., node_color=...)

 ## Task 2.2 - Implementing Graph Neural Networks

 After learning about PyG's data handling, it's time to implement our first Graph Neural Network!

 For this, we will use on of the most simple GNN operators, the **GCN layer** ([Kipf et al. (2017)](https://arxiv.org/abs/1609.02907)), which is defined as

 $$
 \mathbf{x}_v^{(\ell + 1)} = \mathbf{W}^{(\ell + 1)} \sum_{w \in \mathcal{N}(v) \, \cup \, \{ v \}} \frac{1}{c_{w,v}} \cdot \mathbf{x}_w^{(\ell)}
 $$

 where $\mathbf{W}^{(\ell + 1)}$ denotes a trainable weight matrix of shape `[num_output_features, num_input_features]` and $c_{w,v}$ refers to a fixed normalization coefficient for each edge.

 PyG implements this layer via [`GCNConv`](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.GCNConv), which can be executed by passing in the node feature representation `x` and the COO graph connectivity representation `edge_index`.

 With this, we are ready to create our first Graph Neural Network by defining our network architecture in a `torch.nn.Module` class:

In [ ]:
import torch
from torch.nn import Linear
from torch_geometric.nn import GCNConv


class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        torch.manual_seed(1234)
        self.conv1 = GCNConv(dataset.num_features, 4)
        self.conv2 = GCNConv(4, 4)
        self.conv3 = GCNConv(4, 2)
        self.classifier = Linear(2, dataset.num_classes)

    def forward(self, x, edge_index):
        h = self.conv1(x, edge_index)
        h = h.tanh()
        h = self.conv2(h, edge_index)
        h = h.tanh()
        h = self.conv3(h, edge_index)
        h = h.tanh()  # Final GNN embedding space.
        # Apply a final (linear) classifier.
        out = self.classifier(h)
        return out, h

model = GCN()
print(model)

 Here, we first initialize all of our building blocks in `__init__` and define the computation flow of our network in `forward`.
 We first define and stack **three graph convolution layers**, which corresponds to aggregating 3-hop neighborhood information around each node (all nodes up to 3 "hops" away).
 In addition, the `GCNConv` layers reduce the node feature dimensionality to $2$, *i.e.*, $34 \rightarrow 4 \rightarrow 4 \rightarrow 2$. Each `GCNConv` layer is enhanced by a [tanh](https://pytorch.org/docs/stable/generated/torch.nn.Tanh.html?highlight=tanh#torch.nn.Tanh) non-linearity.

 After that, we apply a single linear transformation ([`torch.nn.Linear`](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html?highlight=linear#torch.nn.Linear)) that acts as a classifier to map our nodes to 1 out of the 4 classes/communities.

 We return both the output of the final classifier as well as the final node embeddings produced by our GNN.
 We proceed to initialize our final model via `GCN()`, and printing our model produces a summary of all its used sub-modules.


 ### Embedding the Karate Club Network

 Let's take a look at the node embeddings produced by our GNN before training it.
 Pass in the initial node features `x` and the graph connectivity information `edge_index` to the model, and visualize its 2-dimensional embedding.

In [ ]:
import matplotlib.pyplot as plt
def visualize_embedding(h, color, epoch=None, loss=None):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])
    h = h.detach().cpu().numpy()
    plt.scatter(h[:, 0], h[:, 1], s=140, c=color, cmap="Set2")
    if epoch is not None and loss is not None:
        plt.xlabel(f'Epoch: {epoch}, Loss: {loss.item():.4f}', fontsize=16)
    plt.show()

model = GCN()

embedding= ...
visualize_embedding(embedding, color=data.y)

 Remarkably, even before training the weights of our model, the model produces an embedding of nodes that closely resembles the community-structure of the graph.
 Nodes of the same color (community) are already closely clustered together in the embedding space, although the weights of our model are initialized **completely at random** and we have not yet performed any training so far!
 This leads to the conclusion that GNNs introduce a strong inductive bias, leading to similar embeddings for nodes that are close to each other in the input graph.

 ## Task 2.3 Training on the Karate Club Network

 But can we do better? Let's look at an example on how to train our network parameters based on the knowledge of the community assignments of 4 nodes in the graph (one for each community):

 Since everything in our model is differentiable and parameterized, we can add some labels, train the model and observe how the embeddings react.
 Here, we make use of a semi-supervised or transductive learning procedure: We simply train against one node per class, but are allowed to make use of the complete input graph data.

 Training our model is very similar to any other PyTorch model.
 In addition to defining our network architecture, we define a loss critertion (here, [`CrossEntropyLoss`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)) and initialize a stochastic gradient optimizer (here, [`Adam`](https://pytorch.org/docs/stable/optim.html?highlight=adam#torch.optim.Adam)).
 After that, we perform multiple rounds of optimization, where each round consists of a forward and backward pass to compute the gradients of our model parameters w.r.t. to the loss derived from the forward pass.


 Note that our semi-supervised learning scenario is achieved by the following line:
 ```
 loss = criterion(out[data.train_mask], data.y[data.train_mask])
 ```
 While we compute node embeddings for all of our nodes, we **only make use of the training nodes for computing the loss**.
 Here, this is implemented by filtering the output of the classifier `out` and ground-truth labels `data.y` to only contain the nodes in the `train_mask`.

 Let us now start training and see how our node embeddings evolve over time (best experienced by explicitely running the code):

In [ ]:
import time
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 430})'''))

model = GCN()
criterion = torch.nn.CrossEntropyLoss()  # Define loss criterion.
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  # Define optimizer.

def train(data):
    optimizer.zero_grad()  # Clear gradients.
    out, h = model(data.x, data.edge_index)  # Perform a single forward pass.
    loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.
    return loss, h

for epoch in range(401):
    loss, h = train(data)
    if epoch % 100 == 0:
        visualize_embedding(h, color=data.y, epoch=epoch, loss=loss)

 What do you notice when looking at the embedding?

 Adapted from https://github.com/DeepLearningForPhysicsResearchBook/deep-learning-physics/blob/main/Exercise_10_1.ipynb
 ## 3 Signal Classification using Dynamic Graph Convolutional Neural Networks
 After a long journey through the universe before reaching the earth, the cosmic particles interact with the galactic magnetic field $B$.
 As these particles carry a charge $q$ they are deflected in the field by the Lorentz force $F = q \cdot v × B$.
 Sources of cosmic particles are located all over the sky, thus arrival distributions of the cosmic particles are isotropic in general. However, particles originating from the same source generate on top of the isotropic
 arrival directions, street-like patterns from galactic magnetic field deflections.

 In this tasks we want to classify whether a simulated set of $500$ arriving cosmic particles contains street-like patterns (signal), or originates from an isotropic background.

 Training graph networks can be computationally demanding, thus, we recommend to use a GPU for this task.

In [ ]:
import torch
from torch import nn
from torch_geometric.data import Data, Batch
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
from utils import CosmicRayDS

ds = CosmicRayDS(".")
n_test = 10000
ds_train, ds_test = ds[:-n_test], ds[-n_test:]

 ## Task 3.1
 Extract a single event from the test dataset and inspect it.
 Plot an example sky map using the `skymap` function from `utils`

In [ ]:
from utils import skymap
event0= ...
fig = skymap(..., c=event0.x, zlabel="Energy (normed)", title="Event 0")

 ## Task 3.2
 Generate edges for the event using `knn_graph`.
 Plot the edges by passing the `edge_index` to the `skymap` function. How does the number of edges scale with the $k$?

In [ ]:
from torch_geometric.nn import knn_graph


fig = skymap(
    ...,
    c=...,
    edge_index=...,
    zlabel="Energy (normed)",
    title="Event 0",
)


## Task 3.3
Write a class to return a simple Feed-Forward-Network (FFN) for a given number inputs and outputs. (3 layers, 20 hidden nodes, BatchNorm, LeakyReLU)
The final layer has neither activation nor norm.

In [ ]:
class FFN(nn.Module):
    def __init__(self, n_in, n_out, n_hidden=20):
        super().__init__()
        self.seq = nn.Sequential(
            ...
        )

    def forward(self, *args, **kwargs):
        return self.seq(*args, **kwargs)

 ## Task 3.4
 GNNs classifiers are frequently build in a two step process: First MessagePassingLayers( aka Graph [Convolutional Layers](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#convolutional-layers) ) update the nodes. These exploit the local information. Then, the nodes are aggregated using [Pooling Layers](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#pooling-layers), reducing the graph to a single feature vector. This feature vector is then passed through a FFN to get the classification output.
 Have a look at the documentation of [EdgeConv](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.EdgeConv) and [DynamicEdgeConv](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.DynamicEdgeConv).
 What is the difference?
 -> `EdgeConv` requires a `edge_index` while `DynamicEdgeConv` constructs the `edge_index` on the feature space.
 What the input space of the `nn` passed to EdgeConv?
 -> 2* num_features
 Implement a GNN class with three MPL (not MLP!) using EdgeConv
 and DynamicEdgeConv. For the first MPL, we want to construct
 the `edge_index` on the feature space.
 Use both the energies of the particles (`batch.x`) as well as their positions (`batch.pos`) as an input to the first MPL.
 For the other two layer we may (or may not) choose to construct the `edge_index` on the feature space.
 > Sidenote: Running `knn` multiple times per forward-pass might be quite expensive, depending on the number of nodes and the dimensionality of the space.
 After the MPLs apply a `global_X_pool` and pass the result through a FFN projecting to a single node.

In [ ]:
from torch_geometric.nn import knn_graph, EdgeConv, DynamicEdgeConv, global_add_pool


class GNN(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.conv1 = ...

    def forward(self, batch: Batch):
        # We run knn on the positions
        # knn needs to know about the batches, otherwise it connects
        # points from different events
        edge_index = ...
        x = torch.hstack([batch.x, batch.pos])
        ...
        return x.squeeze()

 ## Task 3.5
 Fill in  the gaps to implement a training loop.
 > The [`BCEWithLogitsLoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html) is recommended as it combines a Sigmoid layer and the `BCELoss`.

In [ ]:
from torch_geometric.loader import DataLoader

device = torch.device("cuda")
loader = DataLoader(ds_train, batch_size=64, shuffle=True)
model = GNN().to(device)
optim = ...
loss_f = nn.BCEWithLogitsLoss()

for iepoch in range(?):
    for batch in tqdm(loader):
        ...

 ## Task 3.6
 Collect the outputs for the test set.

In [ ]:
loader = DataLoader(ds_test, batch_size=64, shuffle=True)
model.eval()
output_list = []
with torch.no_grad():
    for batch in tqdm(loader):
        ...
ytrue, yhat = torch.hstack(output_list).cpu().numpy()

 ## Task 3.7
 Evalutate the model performance on the test set by computing the AUC.

In [ ]:
from sklearn.metrics import roc_auc_score
...

 ## Task 3.8 - Bonus/Open end
 Optimize the model for AUC and speed.